# OMERO Image Annotation Workflow with CellPose
This notebook guides you through annotating images in OMERO using the CellPose workflow. Each step is explained clearly.

## 1. Setup and Installation
Install and import the required libraries. Make sure you have access to OMERO and the omero-annotate-ai package.

In [ ]:
# Import the main package and check dependencies
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
 )

import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")
try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Not available. Install with: pip install -e .[omero]")

## 2. Connect to OMERO
Use the widget below to connect to your OMERO server. Fill in your server details and test the connection before proceeding.

In [ ]:
# Create and display the OMERO connection widget
print("OMERO Connection Setup")
conn_widget = create_omero_connection_widget()
conn_widget.display()
print("\nAfter connecting, run the next cell to continue.")

In [ ]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established. Please use the widget above to connect.")

print("OMERO connection established.")

## 3. Configure the Annotation Workflow
Set up your annotation workflow using the widget below. Select your working directory, OMERO container, and configure annotation parameters.

In [ ]:
# Create and display the workflow widget
print("Annotation Workflow Setup")
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()
print("\nAfter completing the workflow, run the next cell.")

## 4. Review and Run the Annotation Pipeline
Preview your configuration, validate your OMERO selection, and run the annotation pipeline.

#### 4.1. Configuration summary

In [ ]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

print("Configuration Summary:")
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.name}")
print(f"Model: {config.ai_model.model_type}")
print(f"Channel: {config.spatial_coverage.channels}")
print(f"Output: {config.output.output_directory}")
print(f"Resume from Table: {config.workflow.resume_from_table}")
print(f"Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})")

if config.spatial_coverage.three_d:
    print("3D processing: Enabled")

#### 4.2. Convert configuration to an annotation pipeline

In [ ]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"Validating {container_type} with ID {container_id}...")
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")
print(f"Found {container_type}: {container.getName()}")

#### 4.3. Run the annotation pipeline on the files
For CellPose, this will export the images for training into the local folder.

In [ ]:
# Run the pipeline for CellPose
print("Starting annotation pipeline...")
try:
    table_id, processed_images = pipeline.run_cp_workflow()
    print("Images successfully loaded!")
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()

#### 4.4. Run the CellPose GUI to train on the images

In [ ]:
# Run CellPose and finetune the model as needed
# Example: run CellPose from the command line or using the GUI
# pixi run cellpose

#### 4.5. Collect the annotations and upload to OMERO to connect to the original data

#### 4.6. Export as bioimage.io model
For details, see: https://cellpose.readthedocs.io/en/latest/models.html#sharing-models-on-bioimage-io

## 5. Cleanup (Recommended)
Close the OMERO connection when you are finished. This is good practice, especially when working with shared servers.

In [ ]:
# Close OMERO connection (optional, but recommended)
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")